# Intra-lingual A2A VC with S3PRL; S3PRL-VC
[![Generic badge](https://img.shields.io/badge/GitHub-s3plr-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/s3prl
[notebook]:https://colab.research.google.com/github/tarepan/s3prl/blob/master/s3prl/downstream/a2a-vc-vctk/training.ipynb
[tarepan]:https://github.com/tarepan

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!python --version
!pip show torch | sed '2!d'
!/usr/local/cuda/bin/nvcc --version | sed '4!d'

## Setup

Mount GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Clone the `tarepan/s3plr` repository and install dependencies

In [ ]:
!git clone https://github.com/tarepan/s3prl.git

%cd s3prl

# !pip install "torch==1.10.0" -q      # Based on your PyTorch environment
# !pip install "torchaudio==0.10.0" -q # Based on your PyTorch environment

!apt-get install sox
!pip install -e ./   # Repository itself
!pip install fairseq

%cd ./s3prl/downstream/a2a-vc-vctk

!pip install -r requirements.txt

Data preparation

In [ ]:
%cd data

# Clone VCC2020 repository and extract contents
!./vcc2020_download.sh vcc2020/

# Get VCTK-Corpus and extract contents
# !./vctk_download.sh ./ # Replaced because we have VCTK mirror archive
!cp /content/gdrive/MyDrive/ML_data/corpuses/VCTK/ver0_92/archive/VCTK-Corpus.tar.gz "."
!tar xzf ./VCTK-Corpus.tar.gz
!rm ./VCTK-Corpus.tar.gz
%cd ../

# Get pre-trained HiFi-GAN checkpoint archive and extract contents
!./vocoder_download.sh ./

## Preprocessing

## Training

Preprocessing is included in training scripts

In [ ]:
# @./s3prl
%cd ../..
!mkdir -p result/downstream/a2a_vc_vctk_default_vq_wav2vec

# For training resume

## Copy Preprocessed embeddings from storage
# !cp /content/gdrive/MyDrive/ML_data/embedding_VCTK.tar.gz "."
# !tar -zxvf embedding_VCTK.tar.gz
# !cp /content/gdrive/MyDrive/ML_data/stats_VCTK.tar.gz "."
# !tar -zxvf stats_VCTK.tar.gz

## Copy checkpoint from storage
# !cp /content/gdrive/MyDrive/ML_results/S3PRL_VC/states-XXXXX.ckpt result/downstream/a2a_vc_vctk_default_vq_wav2vec


# Training

## Replaced because this script write logs to file, not console
# !./downstream/a2a-vc-vctk/vc_train.sh vq_wav2vec downstream/a2a-vc-vctk/config_ar_taco2.yaml default
## From scratch
!python run_downstream.py -m train --config downstream/a2a-vc-vctk/config_ar_taco2.yaml -p result/downstream/a2a_vc_vctk_default_vq_wav2vec -u vq_wav2vec -d a2a-vc-vctk
## From checkpoint
# !python run_downstream.py -m train --config downstream/a2a-vc-vctk/config_ar_taco2.yaml -p result/downstream/a2a_vc_vctk_default_vq_wav2vec -u vq_wav2vec -d a2a-vc-vctk -e result/downstream/a2a_vc_vctk_default_vq_wav2vec/states-50000.ckpt

# Copy checkpoints
!cp result/downstream/a2a_vc_vctk_default_vq_wav2vec/states-XXXXX.ckpt /content/gdrive/MyDrive/ML_results/S3PRL_VC

## Evaluation

Synthesize waveforms from already generated spectrograms and objectively evaluate them.

In [ ]:
# Waveforms will be properly synthesized and saved, but objective evaluation will failed.
!./downstream/a2a-vc-vctk/decode.sh ./downstream/a2a-vc-vctk/hifigan_vctk result/downstream/a2a_vc_vctk_default_vq_wav2vec/50001

In [ ]:
# Only evaluation (not work now)
# !python downstream/a2a-vc-vctk/evaluate.py --wavdir ./result/downstream/a2a_vc_vctk_default_vq_wav2vec/50001/hifigan_wav --samples 1 --task task1  --data_root ./downstream/a2a-vc-vctk/data

In [ ]:
# # Launch TensorBoard
# %load_ext tensorboard
# %tensorboard --logdir gdrive/MyDrive/ML_results/S3PRL_VC

In [ ]:
# # Usage stat
# ## GPU
# !nvidia-smi -l 3
# ## CPU
# !vmstat 5
# !top